In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
from keras import layers

In [3]:
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy

In [4]:
from openpyxl import Workbook
from openpyxl import load_workbook

In [5]:
import pathlib
from datetime import datetime

In [6]:
import itertools
import os

In [7]:
(train_ds, validation_ds, test_ds), info = tfds.load(
    "colorectal_histology",
    split=["train[:80%]", "train[80%:85%]", "train[85%:]"],
    with_info=True,
    as_supervised=True,
    shuffle_files= True,
)

num_classes = info.features['label'].num_classes

In [8]:
size = (150, 150)
batch_size = 16

train_ds = train_ds.map(lambda x, y: (tf.image.resize(x, size), y))
validation_ds = validation_ds.map(lambda x, y: (tf.image.resize(x, size), y))
test_ds = test_ds.map(lambda x, y: (tf.image.resize(x, size), y))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [9]:
train_ds = train_ds.cache().batch(batch_size).prefetch(buffer_size=1)
validation_ds = validation_ds.cache().batch(batch_size).prefetch(buffer_size=1)
test_ds = test_ds.cache().batch(batch_size).prefetch(buffer_size=1)

In [10]:
%cd  /home/reynier/py_projects/Deployments/functions

/home/reynier/py_projects/Deployments/functions


In [11]:
from Nets import GetNeuralNetworkModel
from Stats import WeightQuantization, ActivationStats, CheckAccuracyAndLoss, QuantizationEffect, GetReadAndWrites
from FileAnalize import analize_file, analize_file_uno,analize_file_uno_ceros, save_file, load_file
from funciones import buffer_vectores
from Simulation import buffer_simulation, save_obj, load_obj

In [12]:
model = GetNeuralNetworkModel('Xception',(150,150,3),8, quantization = False, aging_active=False)

In [13]:
model.compile(
    optimizer=Adam(),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

In [14]:
%cd  /home/reynier/py_projects/Deployments/Xception/Weights

/home/reynier/py_projects/Deployments/Xception/Weights


In [15]:
model.load_weights('weights.data')

In [16]:
Image_item = []
Predict = []

vol=54
for i in range(1):

    %cd '/home/reynier/py_projects/Deployments/Data/byte_x/VC_707'
    error_mask = load_obj('error_mask_0' + str(vol))
    locs = load_obj('locs_0' + str(vol))

    vol = vol + 1
    
    model_refered = GetNeuralNetworkModel('Xception', (150,150,3), 8, faulty_addresses=locs, masked_faults=error_mask, 
                                          aging_active=True, word_size=16, frac_size=7, batch_size=batch_size)    

    model_refered.compile(
        optimizer=Adam(),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])

    %cd  /home/reynier/py_projects/Deployments/Xception/Weights

    model_refered.load_weights('weights.data')
    iterator = iter(test_ds)
    
    index = 1
    while index <= len(test_ds):
        image = next(iterator)[0]    

        image_batch = image.numpy()
        predicted_batch_refered = model_refered.predict(image_batch)
        predicted_batch_normal = model.predict(image_batch)
        diff_nets_sum = np.sum(tf.math.abs(tf.math.subtract(predicted_batch_normal, predicted_batch_refered)))

        Predict.append(diff_nets_sum)
        Image_item.append(index)
        index = index + 1

Predict_DF=pd.DataFrame(Predict)
Image_item_DF=pd.DataFrame(Image_item)

comparative_values = pd.concat([Predict_DF, Image_item_DF], axis=1, join='outer')
comparative_values.columns =['Prediction','Image_item']
comparative_values.to_excel('Xception_comparativa_fichero_byte_x.xlsx', sheet_name='report', index=False)

print(' completada: ', datetime.now().strftime("%H:%M:%S"))
print(' completada: ', datetime.now().strftime("%H:%M:%S"))

/home/reynier/py_projects/Deployments/Data/byte_x/VC_707
/home/reynier/py_projects/Deployments/Xception/Weights


/home/reynier/anaconda3/envs/tf_ml_projects/lib/python3.9/site-packages/tensorflow/python/data/ops/structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


1/1 [==============================] - 0s 94ms/step
 completada:  06:32:18
 completada:  06:32:18


In [17]:
Image_item = []
Predict = []

vol=54
for i in range(1):

    %cd '/home/reynier/py_projects/Deployments/Data/ECC/VC_707'
    error_mask = load_obj('error_mask_0' + str(vol))
    locs = load_obj('locs_0' + str(vol))

    vol = vol + 1
    
    model_refered = GetNeuralNetworkModel('Xception', (150,150,3), 8, faulty_addresses=locs, masked_faults=error_mask, 
                                          aging_active=True, word_size=16, frac_size=7, batch_size=batch_size)    

    model_refered.compile(
        optimizer=Adam(),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])

    %cd  /home/reynier/py_projects/Deployments/Xception/Weights

    model_refered.load_weights('weights.data')
    iterator = iter(test_ds)
    
    index = 1
    while index <= len(test_ds):
        image = next(iterator)[0]    

        image_batch = image.numpy()
        predicted_batch_refered = model_refered.predict(image_batch)
        predicted_batch_normal = model.predict(image_batch)        
        diff_nets_sum = np.sum(tf.math.abs(tf.math.subtract(predicted_batch_normal, predicted_batch_refered)))

        Predict.append(diff_nets_sum)
        Image_item.append(index)
        index = index + 1

Predict_DF=pd.DataFrame(Predict)
Image_item_DF=pd.DataFrame(Image_item)

comparative_values = pd.concat([Predict_DF, Image_item_DF], axis=1, join='outer')
comparative_values.columns =['Prediction','Image_item']
comparative_values.to_excel('Xception_comparativa_fichero_ECC.xlsx', sheet_name='report', index=False)

print(' completada: ', datetime.now().strftime("%H:%M:%S"))
print(' completada: ', datetime.now().strftime("%H:%M:%S"))

/home/reynier/py_projects/Deployments/Data/ECC/VC_707
/home/reynier/py_projects/Deployments/Xception/Weights


/home/reynier/anaconda3/envs/tf_ml_projects/lib/python3.9/site-packages/tensorflow/python/data/ops/structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


1/1 [==============================] - 0s 87ms/step
 completada:  06:32:51
 completada:  06:32:51


In [18]:
Image_item = []
Predict = []

vol=54
for i in range(1):

    %cd '/home/reynier/py_projects/Deployments/Data/ECCx/VC_707'
    error_mask = load_obj('error_mask_0' + str(vol))
    locs = load_obj('locs_0' + str(vol))

    vol = vol + 1
    
    model_refered = GetNeuralNetworkModel('Xception', (150,150,3), 8, faulty_addresses=locs, masked_faults=error_mask, 
                                          aging_active=True, word_size=16, frac_size=7, batch_size=batch_size)    

    model_refered.compile(
        optimizer=Adam(),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])

    %cd  /home/reynier/py_projects/Deployments/Xception/Weights

    model_refered.load_weights('weights.data')
    iterator = iter(test_ds)
    
    index = 1
    while index <= len(test_ds):
        image = next(iterator)[0]    

        image_batch = image.numpy()
        predicted_batch_refered = model_refered.predict(image_batch)
        predicted_batch_normal = model.predict(image_batch)        
        diff_nets_sum = np.sum(tf.math.abs(tf.math.subtract(predicted_batch_normal, predicted_batch_refered)))

        Predict.append(diff_nets_sum)
        Image_item.append(index)
        index = index + 1

Predict_DF=pd.DataFrame(Predict)
Image_item_DF=pd.DataFrame(Image_item)

comparative_values = pd.concat([Predict_DF, Image_item_DF], axis=1, join='outer')
comparative_values.columns =['Prediction','Image_item']
comparative_values.to_excel('Xception_comparativa_fichero_ECCx.xlsx', sheet_name='report', index=False)

print(' completada: ', datetime.now().strftime("%H:%M:%S"))
print(' completada: ', datetime.now().strftime("%H:%M:%S"))

/home/reynier/py_projects/Deployments/Data/ECCx/VC_707


/home/reynier/py_projects/Deployments/Xception/Weights


/home/reynier/anaconda3/envs/tf_ml_projects/lib/python3.9/site-packages/tensorflow/python/data/ops/structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


1/1 [==============================] - 0s 90ms/step
 completada:  06:33:24
 completada:  06:33:24


In [19]:
Image_item = []
Predict = []

vol=54
for i in range(1):

    %cd '/home/reynier/py_projects/Deployments/Data/mask_volteada/VC_707'
    error_mask = load_obj('error_mask_0' + str(vol))
    locs = load_obj('locs_0' + str(vol))

    vol = vol + 1
    
    model_refered = GetNeuralNetworkModel('Xception', (150,150,3), 8, faulty_addresses=locs, masked_faults=error_mask, 
                                          aging_active=True, word_size=16, frac_size=7, batch_size=batch_size)    

    model_refered.compile(
        optimizer=Adam(),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])

    %cd  /home/reynier/py_projects/Deployments/Xception/Weights

    model_refered.load_weights('weights.data')
    iterator = iter(test_ds)
    
    index = 1
    while index <= len(test_ds):
        image = next(iterator)[0]    

        image_batch = image.numpy()
        predicted_batch_refered = model_refered.predict(image_batch)
        predicted_batch_normal = model.predict(image_batch)        
        diff_nets_sum = np.sum(tf.math.abs(tf.math.subtract(predicted_batch_normal, predicted_batch_refered)))

        Predict.append(diff_nets_sum)
        Image_item.append(index)
        index = index + 1

Predict_DF=pd.DataFrame(Predict)
Image_item_DF=pd.DataFrame(Image_item)

comparative_values = pd.concat([Predict_DF, Image_item_DF], axis=1, join='outer')
comparative_values.columns =['Prediction','Image_item']
comparative_values.to_excel('Xception_comparativa_fichero_mask_volteada.xlsx', sheet_name='report', index=False)

print(' completada: ', datetime.now().strftime("%H:%M:%S"))
print(' completada: ', datetime.now().strftime("%H:%M:%S"))

/home/reynier/py_projects/Deployments/Data/mask_volteada/VC_707


/home/reynier/py_projects/Deployments/Xception/Weights


/home/reynier/anaconda3/envs/tf_ml_projects/lib/python3.9/site-packages/tensorflow/python/data/ops/structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


1/1 [==============================] - 0s 103ms/step


1/1 [==============================] - 0s 117ms/step
 completada:  06:33:58
 completada:  06:33:58


In [20]:
Image_item = []
Predict = []

vol=54
for i in range(1):

    %cd '/home/reynier/py_projects/Deployments/Data/base_line/VC_707'
    error_mask = load_obj('error_mask_0' + str(vol))
    locs = load_obj('locs_0' + str(vol))

    vol = vol + 1
    
    model_refered = GetNeuralNetworkModel('Xception', (150,150,3), 8, faulty_addresses=locs, masked_faults=error_mask, 
                                          aging_active=True, word_size=16, frac_size=7, batch_size=batch_size)    

    model_refered.compile(
        optimizer=Adam(),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])

    %cd  /home/reynier/py_projects/Deployments/Xception/Weights

    model_refered.load_weights('weights.data')
    iterator = iter(test_ds)
    
    index = 1
    while index <= len(test_ds):
        image = next(iterator)[0]    

        image_batch = image.numpy()
        predicted_batch_refered = model_refered.predict(image_batch)
        predicted_batch_normal = model.predict(image_batch)        
        diff_nets_sum = np.sum(tf.math.abs(tf.math.subtract(predicted_batch_normal, predicted_batch_refered)))

        Predict.append(diff_nets_sum)
        Image_item.append(index)
        index = index + 1

Predict_DF=pd.DataFrame(Predict)
Image_item_DF=pd.DataFrame(Image_item)

comparative_values = pd.concat([Predict_DF, Image_item_DF], axis=1, join='outer')
comparative_values.columns =['Prediction','Image_item']
comparative_values.to_excel('Xception_comparativa_fichero_base_line.xlsx', sheet_name='report', index=False)

print(' completada: ', datetime.now().strftime("%H:%M:%S"))
print(' completada: ', datetime.now().strftime("%H:%M:%S"))

/home/reynier/py_projects/Deployments/Data/base_line/VC_707
/home/reynier/py_projects/Deployments/Xception/Weights


/home/reynier/anaconda3/envs/tf_ml_projects/lib/python3.9/site-packages/tensorflow/python/data/ops/structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


1/1 [==============================] - 0s 253ms/step


1/1 [==============================] - 0s 91ms/step
 completada:  06:34:31
 completada:  06:34:31
